# State of veteran employment, 2020
\[_In case of broken visualizations, view notebook here in [nbviewer](https://nbviewer.org/github/atoti/notebooks/blob/master/notebooks/veteran-emp/main.ipynb)._]

In March of 2021, a memo was released by the US Bureau of Labor Statistics summarizing the state of veteran employment during 2020.  The data, sourced from a survey, explored labor statistics through the lens gender, age, race, and veteran service period.  It explored relationships between employment and disability.  The [released memo](https://www.bls.gov/news.release/pdf/vet.pdf) was a thorough (read, dense) 26 pages, full of text and tables.

We explore this dataset in this notebook.  We selected a few  highlights from the memo and summarize those key takeaways via visuals.  We also use those visuals and others to construct a dashboard.

<div style="text-align: center;" ><a href="https://www.atoti.io/?utm_source=gallery&utm_content=veteran-data" target="_blank" rel="noopener noreferrer"><img src="https://data.atoti.io/notebooks/banners/discover.png" alt="Try atoti"></a></div>

## Getting Started

In [1]:
import atoti as tt

In [2]:
session = tt.create_session(config={"user_content_storage": "./content", "port": 9090})

We will use a subset of the underlying data summarized in the aforementioned memo.  All tables related to the data set are available [here](https://www.bls.gov/news.release/vet.nr0.htm) on the [US Bureau of Labor Statistics website](https://www.bls.gov/). 

We'll create a dashboard from the ensuing visuals, hence set our session config to store our content metadata.  This will allow us to save and return to our dashboard later on.

In [3]:
veteran_data = session.read_csv(
    "s3://data.atoti.io/notebooks/veteran-emp/veteran-data.csv"
)

In [4]:
veteran_data.head()

,Veteran status,period of service,sex,age,Civilian noninstitutional population,Total,Percent of population,Total Employed,Percent of population employed,Total Unemployeed,Percent of labor force unemployeed,Not in labor force
0,NONVETERANS,NONVETERANS,m,18 to 24 years,14221,9160,64,7807,55,1353,15,5061
1,NONVETERANS,NONVETERANS,m,25 to 34 years,21140,18441,87,16878,80,1563,8,2699
2,NONVETERANS,NONVETERANS,m,35 to 44 years,18691,16814,90,15769,84,1045,6,1877
3,NONVETERANS,NONVETERANS,m,45 to 54 years,17336,15124,87,14218,82,905,6,2213
4,NONVETERANS,NONVETERANS,m,55 to 64 years,17340,12479,72,11680,67,799,6,4861


In [5]:
cube = session.create_cube(veteran_data, mode="no_measures")

In [6]:
h, l, m = cube.hierarchies, cube.levels, cube.measures

### Adusting levels and hierarchies

Two of the levels in our cube have an inherent relationship between them: *period of service* is very much so related to *Veteran status*--a nonveteran won't have a classification of *period of service*.  We can add the *period of service* level to our Veteran Status hierarchy, and delete the *period of service* hierarchy.

In [7]:
h["Veteran status"] = [l["Veteran status"], l["period of service"]]

In [8]:
del h["period of service"]

In [9]:
h

Hierarchies()

### Adjusting measures

Two things to note in our underlying data:
* it provides percents, which do not make sense to sum or mean, and
* there are certain stats which are referenced but not included as a column--such as *Civilian labor force*, defined as the *Civilian noninstutional population* minus the *not in labor force* population

To address this, we'll delete the unnecessary percent columns, and add this additional metric.

In [10]:
veteran_data.columns

['Veteran status',
 'period of service',
 'sex',
 'age',
 'Civilian noninstitutional population',
 'Total',
 'Percent of population',
 'Total Employed',
 'Percent of population employed',
 'Total Unemployeed',
 'Percent of labor force unemployeed',
 'Not in labor force']

In [11]:
veteran_data["sex"].data_type

DataType(java_type='string', nullable=False)

In [12]:
for column_name in [
    column_name
    for column_name in veteran_data.columns
    if veteran_data[column_name].data_type.java_type != "string"
]:

    cube.measures[f"{column_name}.SUM"] = tt.agg.sum(veteran_data[column_name])

In [13]:
m

Measures()

In [14]:
m["Civilian labor force"] = (
    m["Civilian noninstitutional population.SUM"] - m["Not in labor force.SUM"]
)

## Getting a feel for the breakdowns

In [15]:
session.visualize("Understanding the relative size of service group populations")

Open the notebook in JupyterLab with the atoti extension enabled to build this widget.

In [16]:
session.visualize("Breakdown of employment")

Open the notebook in JupyterLab with the atoti extension enabled to build this widget.

In [17]:
session.visualize("Breakdown of employment - by 100%")

Open the notebook in JupyterLab with the atoti extension enabled to build this widget.

In [18]:
session.visualize("Age based breakdown for nonveterans vs veterans")

Open the notebook in JupyterLab with the atoti extension enabled to build this widget.

In [19]:
session.visualize(
    "Size of civilian noninstitutional population by age, split by veteran status"
)

Open the notebook in JupyterLab with the atoti extension enabled to build this widget.

## Visualizing a few key points

While there are many statistics highlighted in the memo, we'll visualize the following points:

* The rate of unemployment for male veterans was 6.5 percent, little different from the rate of 6.7 percent for female veterans.
* Among the 581,000 unemployed veterans in 2020, 54 percent were ages 25 to 54, 41 percent were age 55 and over, and 5 percent were ages 18 to 24.
* In 2020, 18.5 million men and women were veterans, accounting for about 7 percent of the civilian noninstitutional population age 18 and over
* Of all veterans, 1 in 10 were women. 

In [20]:
session.visualize("Visualizing male vs female veteran unemployment")

Open the notebook in JupyterLab with the atoti extension enabled to build this widget.

In [21]:
session.visualize("Visualizing the age breakdown of unemployeed vets")

Open the notebook in JupyterLab with the atoti extension enabled to build this widget.

In [22]:
session.visualize(
    "Visualizing the veteran segment of the civilian noninstitutional population"
)

Open the notebook in JupyterLab with the atoti extension enabled to build this widget.

In [23]:
session.visualize("Visualizing the proportion of veterans who are women")

Open the notebook in JupyterLab with the atoti extension enabled to build this widget.

From here, we can right publish these visuals, plus any others we liked from our exploration, and create a dashboard.  We can access our dashboard via the below:

In [24]:
session.link(path="/#/dashboard/11d")

Open the notebook in JupyterLab with the atoti extension enabled to see this link.

<div style="text-align: center;" ><a href="https://www.atoti.io/?utm_source=gallery&utm_content=veteran-data" target="_blank" rel="noopener noreferrer"><img src="https://data.atoti.io/notebooks/banners/discover-try.png" alt="Try atoti"></a></div>